<a href="https://colab.research.google.com/github/Blackman9t/Coursera_Capstone/blob/master/applied_data_science_capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Applied Data Science Capstone-Project


### This Capsone-Project is the final course in a series of nine(9) Data Science Specialization courses taught by IBM via Coursera. <br>
###The courses include the following:-
1. [What is Data Science?](https://www.coursera.org/learn/what-is-datascience/home/welcome)
2. [Open Source tools for Data Science](https://www.coursera.org/learn/open-source-tools-for-data-science/home/welcome)
3. [Python for Data Science](https://www.coursera.org/learn/python-for-applied-data-science/home/welcome)
4. [Data Science Methodology](https://www.coursera.org/learn/data-science-methodology/home/welcome)
5. [Databases and SQL for Data Science](https://www.coursera.org/learn/sql-data-science/home/welcome)
6. [Data Visualization with Python](https://www.coursera.org/learn/python-for-data-visualization/home/welcome)
7. [Data Analysis with Python](https://www.coursera.org/learn/data-analysis-with-python/home/welcome)
8. [Machine Learning with Python](https://www.coursera.org/learn/machine-learning-with-python/home/welcome)
9. [Applied Data Science Capstone](https://www.coursera.org/learn/applied-data-science-capstone/home/welcome)

#### It is mandatory that all courses from 1 to 8 are passed with acceptable grades before attempting this Capsone-Project.

Importing requred libraries

In [0]:
import pandas as pd
import numpy as np

In [170]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<h2>Problem Description</h2>

<h3>The problem I want to solve using Geo-location Data is helping a new Tech-StartUp find the ideal location for its office in The City of Lagos Nigeria.
This office should be situated in a good location that has the following:-</h3><br>

Central location.<br>
Close proximity to Tech-Hubs and talent.<br>
High feet traffic area for easy interaction with potential customers.<br>
Nearness to Educational institutions for research and interactions with academics.<br>
Nearness to cafes and restaurants for business meetings or lunch-meets.<br>
Nearness to bus-stops, seaports and airports.<br>
Security and safety.<br>
Cluster of economic activities and similar or complementing businesses.


<h2>Background to Problem</h2>

I have lived in Lagos for some time off and on, longest has been 3 years stretch from 2006 to 2009. <br>
I have also done some travelling to other countries such as Russia, which by the way is the largest country in The World by land mass, South-Africa, Ghana, Dubai, Mauritius and a few others.<br>
By the way, Mauritius is The wealthiest country in Africa with average wealth per person increasing from $(21700 - 25700) within one year recently ([link](https://www.graphic.com.gh/business/business-news/accra-ranked-10th-wealthiest-city-in-africa.html)).<br>
My analysis today is on Lagos state, which is the 4th wealthiest city in Africa, fourth place behind Johannesburg, Cairo and Cape Town ([link](https://www.graphic.com.gh/business/business-news/accra-ranked-10th-wealthiest-city-in-africa.html)).<br>
Lagos city is home to over 6,800 $$ millionaires, 370 $$ multi-millionaires and 4 $$ billionaires.<br>
Lagos is also the commercial capital of Nigeria (The Most populous black nation), and the commercial hub of West Africa.<br>
I have only recently returned to Lagos city, got enrolled and completed The [Founder Institute](https://fi.co/) programme and currently building a Startup in The Insuretech industry ([link](http://bit.ly/escrowallet)).<br>
Therefore I want to use this opportunity to help future Startups find the most ideal location in Lagos city, for an office.

<h2>Data Requirements and Overview</h2>

**The Dataset is the wikipedia page of Lagos state** see [link](https://en.wikipedia.org/wiki/List_of_Lagos_State_local_government_areas_by_population)

We shall explore Lagos city through its respective Local Government Areas or Boroughs via this [link](https://en.wikipedia.org/wiki/List_of_Lagos_State_local_government_areas_by_population).<br>
The above link is a web page that shows the respective Local Government Areas in Lagos State and each population figure. It's a public wikipedia data page<br>
This data will be analysed through the following steps:-<br>
1. We shall scrape the web page using the beautiful soup library<br>
2. We shall use The Foresquare API calls to retrieve geo-location data<br>
3. We shall fetch the text data using the requests library<br>
4. We shall convert it from JSON to Pandas dataframe using the josn_normalize module<br>
5. We shall use The folium library to view the maps and plot these via matplotlib library.<br>
Then we shall cluster Lagos City into its respective Postcodes and Local Government Areas and analyse each area for its appeal as a location for a Tech-Startup based on the aforementioned parameters.<br>
After this we shall select our Top-Location.<br>
5. Finally to add some fun, we shall use the Word-cloud library to display the names of the top hubs, restaurants, offices and education institutes in our Top-Location.

**So please sit back, relax and enjoy the ride as we explore Lagos City**

First let's import required libraries

In [171]:
from bs4 import BeautifulSoup
import requests  # library to handle requests
import pandas as pd
import json  # library to handle JSON files
from pandas.io.json import json_normalize  # transform json files to pandas dataframes
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import numpy as np
import csv

print('All modules imported')

All modules imported


In [0]:
# Lets save the weblink for The Lagos Data

lagos_link = 'https://en.wikipedia.org/wiki/List_of_Lagos_State_local_government_areas_by_population'

Get the source code html data from the website

In [0]:
source = requests.get(lagos_link).text

Lets Use BeautifulSoup to parse it

In [0]:
soup = BeautifulSoup(source,'lxml')

# Let's pretty print it in its right shape

#print(soup.prettify())

Next let's get the table that contains the data we want to scrape

In [0]:
my_table = soup.find_all('td')
#mw-content-text
# Let's view the table

# my_table

Next let's iterate through each link and append the text parts

In [176]:
type(my_table)

bs4.element.ResultSet

In [0]:
table_text = []

for data in my_table:
    table_text.append(data.text)

In [0]:
# Let's see the text

# table_text

First let's extract only the relevent data from the table

In [179]:
relevant_table_data = table_text[4:-3]

# Next let's see the first 3 elements
relevant_table_data[:3]

['1\n', 'Alimosho\n', '5,700,714\n']

Next lets create a dictionary and append the LGA and corresponding Population data to it

In [0]:
table_dict={'LGA':[], 'POP':[]}
count = 0

for item in relevant_table_data:
    # First let's strip off the \n at the end
    item = item.strip('\n')
    try:
        item = int(item)
    except:
        # if second item after the int, append to POP
        if count > 0:
            # First let's remove the commas
            item = item.replace(',','')
            # Next let's convert to an integer so we can use it for calculations
            item = int(item)
            # Finally, let's append it to the Population list of the dictionary
            table_dict['POP'].append(item)
            count = 0
        else:
         # if first item after the int, append to LGA
            table_dict['LGA'].append(item)
            count +=1
            

Next let's create a Dataframe of each LGA and its respective Population

In [181]:
lagos_df = pd.DataFrame(table_dict)

# Let's see the first five rows
lagos_df.head()

,LGA,POP
0,Alimosho,5700714
1,Ajeromi-Ifelodun,1746634
2,Kosofe,665998
3,"Mushin, Lagos",633543
4,Oshodi-Isolo,621789


Appending the Latitude and Longitude Values:<br>
First let's define a simple method to extract each LGA or Borough Lats and Longs data

In [0]:
def latitude_longitude(Borough):
    import time
    """ Method takes a Series object and returns
    a list of Latitude and corresponding Longitude data,
    using the geopy library.
    This method also prints out the coordinate data"""
    
    address = str(Borough)
    
    # We must define a geolocator user agent
    geolocator = Nominatim(user_agent="NG_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are lat {} and long {}.'.format(address, latitude, longitude))
    time.sleep(2)  # we let 2 secs pass after calling each location lat/lon so that the geocode function does not crash
    return [latitude, longitude]


In [183]:
lagos_df['latitude'] = lagos_df['LGA'].apply(latitude_longitude)

The geograpical coordinates of Alimosho are lat 6.58434295 and long 3.25763109723237.
The geograpical coordinates of Ajeromi-Ifelodun are lat 6.4551217 and long 3.33594625182987.
The geograpical coordinates of Kosofe are lat 6.581974 and long 3.41483594161784.
The geograpical coordinates of Mushin, Lagos are lat 6.53263245 and long 3.35202197179877.
The geograpical coordinates of Oshodi-Isolo are lat 6.54001 and long 3.31241461318394.
The geograpical coordinates of Ojo are lat 1.0958392 and long -76.6110685.
The geograpical coordinates of Ikorodu are lat 6.6191233 and long 3.5041271.
The geograpical coordinates of Surulere are lat 6.48720145 and long 3.35325872672398.
The geograpical coordinates of Agege are lat 6.6252564 and long 3.31120926770265.
The geograpical coordinates of Ifako-Ijaiye are lat 6.6636025 and long 3.289491.
The geograpical coordinates of Somolu are lat 6.583333 and long 3.466667.
The geograpical coordinates of Amuwo-Odofin are lat 6.4470234 and long 3.2662797116825

Let's see the data frame again

In [184]:
lagos_df.head()

,LGA,POP,latitude
0,Alimosho,5700714,"[6.58434295, 3.25763109723237]"
1,Ajeromi-Ifelodun,1746634,"[6.4551217, 3.33594625182987]"
2,Kosofe,665998,"[6.581974, 3.41483594161784]"
3,"Mushin, Lagos",633543,"[6.53263245, 3.35202197179877]"
4,Oshodi-Isolo,621789,"[6.54001, 3.31241461318394]"


Next let's loop through the data frame and separate Latitude from Longitude and make lat and lons just numbers not lists


In [0]:
lon_list = []
for i, j in lagos_df.iterrows():
    lon_list.append(j.latitude[1])
    lagos_df.iat[i,2] = j.latitude[0]
      
# next let's assign the lon_list as the value of the Longitude Column

lagos_df['longitude'] = lon_list

Let's see our table with the separated latitude and longitude values

In [186]:
lagos_df.head()

,LGA,POP,latitude,longitude
0,Alimosho,5700714,6.58434,3.257631
1,Ajeromi-Ifelodun,1746634,6.45512,3.335946
2,Kosofe,665998,6.58197,3.414836
3,"Mushin, Lagos",633543,6.53263,3.352022
4,Oshodi-Isolo,621789,6.54001,3.312415


Okay so at this point let's look at each Borough or L.G.A closer. Let's start with the first one and visualize it via Folium library

Let's get the geographic coordinates of Alimosho, the first L.G.A

In [187]:
alimosho_data = lagos_df.loc[0,].to_frame().T
alimosho_data

,LGA,POP,latitude,longitude
0,Alimosho,5700714,6.58434,3.25763


In [188]:
lat, lon = latitude_longitude(alimosho_data.iloc[0,0])

print('lat: {}, lon: {}'.format(lat,lon))

The geograpical coordinates of Alimosho are lat 6.58434295 and long 3.25763109723237.
lat: 6.58434295, lon: 3.25763109723237


In [189]:
# create map of Alimosho using latitude and longitude values
map_alimosho = folium.Map(location=[lat, lon], zoom_start=14)

# add markers to map

label = "{}".format('Alimosho')
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [lat, lon],
    radius=10,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.8,
    parse_html=False).add_to(map_alimosho)  

map_alimosho  # Feel free to zoom in to see more

Now Let's explore the data using foursquare API

In [190]:
CLIENT_ID = 'L2XUYCRD44VP2YBDMZ3XY0Z4XKO1JODM5SMG3IIYRZ5330FD' # your Foursquare ID
CLIENT_SECRET = '5KI00FG4RT2VJQTIDPXFY1N4AOKTIXLIZEAO0AHPP3DPD0U1' # your Foursquare Secret
VERSION = '20191212'
LIMIT = 50
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: L2XUYCRD44VP2YBDMZ3XY0Z4XKO1JODM5SMG3IIYRZ5330FD
CLIENT_SECRET:5KI00FG4RT2VJQTIDPXFY1N4AOKTIXLIZEAO0AHPP3DPD0U1


Let's get the full data for Alimosho LGA

Now, let's get the top 200 venues that are in the first entry within a radius of 10 Kilometers.

In [0]:
radius = 10000 # 10,000 meters is 10KM
LIMIT=200

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    alimosho_data.latitude[0], 
    alimosho_data.longitude[0], 
    radius, 
    LIMIT)

let's Send the GET request and examine the resutls

In [192]:
result = requests.get(url).json()

result.keys()

dict_keys(['meta', 'response'])

In [0]:
# show the result
# result

Let's define a function that extracts the category of the venue

Let's save only the venues from result in a variable called venues

In [194]:
venues = result['response']['groups'][0]['items']
len(venues)

# This tells us about 40 venues are returned in a 10 Kilometer radius of Alimosho LGA

39

In [0]:
# See the venues json data

# venues

Next, lets convert the result of nearby alimosho borough venues returned into a DataFrame

In [196]:
nearby_venues = json_normalize(venues)
nearby_venues.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.location.neighborhood,venue.venuePage.id
0,e-0-4e261fce315161d79d2a8fa5-0,0,"[{'summary': 'This spot is popular', 'type': '...",4e261fce315161d79d2a8fa5,SHAWARMA.COME,federal junction beside mr biggs gowon estate ...,6.604609,3.277413,"[{'label': 'display', 'lat': 6.604608729562348...",3142,+234,NG,Lagos,Lagos,Nigeria,[federal junction beside mr biggs gowon estate...,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[],NaN,NaN,NaN
1,e-0-4ecbf689b6346e2d3a4dead5-1,0,"[{'summary': 'This spot is popular', 'type': '...",4ecbf689b6346e2d3a4dead5,Tantalisers,Abule Egba,6.649299,3.265609,"[{'label': 'display', 'lat': 6.649299147190656...",7284,NaN,NG,Lagos,Lagos,Nigeria,"[Abule Egba, Lagos, Lagos, Nigeria]","[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",0,[],NaN,NaN,NaN
2,e-0-4f75a8e7e4b06c81e0e3ba05-2,0,"[{'summary': 'This spot is popular', 'type': '...",4f75a8e7e4b06c81e0e3ba05,Wings Restaurant,NaN,6.576023,3.325612,"[{'label': 'display', 'lat': 6.576022675981682...",7574,NaN,NG,Lagos,Lagos,Nigeria,"[Lagos, Lagos, Nigeria]","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],NaN,NaN,NaN
3,e-0-51176f6ce4b05931e449abbc-3,0,"[{'summary': 'This spot is popular', 'type': '...",51176f6ce4b05931e449abbc,Ipaja market ipaja,Ipaja Last bus stop,6.602248,3.255585,"[{'label': 'display', 'lat': 6.602248491329565...",2006,NaN,NG,Lagos,Lagos,Nigeria,"[Ipaja Last bus stop (Ipaja), Lagos, Lagos, Ni...","[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",0,[],Ipaja,NaN,NaN
4,e-0-4c93872fa25fef3b4c231223-4,0,"[{'summary': 'This spot is popular', 'type': '...",4c93872fa25fef3b4c231223,De Grange suites & bar,"4, association rd, boys town Ipaja",6.602309,3.267038,"[{'label': 'display', 'lat': 6.602309210296299...",2254,NaN,NG,Lagos,Lagos,Nigeria,"[4, association rd, boys town Ipaja (btw ipaja...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",0,[],btw ipaja road and ayobo,NaN,NaN


Okay let's now filter the venue DataFrame and select only the volumns needed 

In [197]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance', 'venue.id']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.distance,venue.id
0,SHAWARMA.COME,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",6.604609,3.277413,3142,4e261fce315161d79d2a8fa5
1,Tantalisers,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",6.649299,3.265609,7284,4ecbf689b6346e2d3a4dead5
2,Wings Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",6.576023,3.325612,7574,4f75a8e7e4b06c81e0e3ba05
3,Ipaja market ipaja,"[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",6.602248,3.255585,2006,51176f6ce4b05931e449abbc
4,De Grange suites & bar,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",6.602309,3.267038,2254,4c93872fa25fef3b4c231223


We can see that the categories column is not well formatted. Let's clean it out.<br>
First let's inspect it closer

In [198]:
nearby_venues['venue.categories'][0]

[{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
   'suffix': '.png'},
  'id': '4bf58dd8d48988d1ca941735',
  'name': 'Pizza Place',
  'pluralName': 'Pizza Places',
  'primary': True,
  'shortName': 'Pizza'}]

Ok so let's now define a simple method that returns only the name for each category.<br>
Then use the apply method to apply it on each category item in the dataframe
                                                            

In [0]:
def clean_category(x):
    x = x[0]['name']
    return x

Now let's apply it

In [0]:
nearby_venues['venue.categories'] = nearby_venues['venue.categories'].apply(clean_category)

Finally let's view the cleaned category column

In [201]:
alimosho_venues = nearby_venues

alimosho_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.distance,venue.id
0,SHAWARMA.COME,Pizza Place,6.604609,3.277413,3142,4e261fce315161d79d2a8fa5
1,Tantalisers,Burger Joint,6.649299,3.265609,7284,4ecbf689b6346e2d3a4dead5
2,Wings Restaurant,Restaurant,6.576023,3.325612,7574,4f75a8e7e4b06c81e0e3ba05
3,Ipaja market ipaja,Market,6.602248,3.255585,2006,51176f6ce4b05931e449abbc
4,De Grange suites & bar,Bar,6.602309,3.267038,2254,4c93872fa25fef3b4c231223


Lets see the different types of venues present

In [202]:
alimosho_cat_count = alimosho_venues.groupby('venue.categories').size().to_frame()

alimosho_cat_count.columns = ['frequency']

# Let's see the categories of venues present in the alimosho LGA
alimosho_cat_count.T

venue.categories,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Athletics & Sports,Bar,Boutique,Burger Joint,Campground,Convenience Store,Department Store,Electronics Store,Farmers Market,Fast Food Restaurant,Heliport,Hotel,Lounge,Market,Park,Pizza Place,Restaurant,Shopping Mall,Soup Place,Wine Shop
frequency,1,1,1,1,1,1,1,1,2,1,1,1,1,1,6,1,2,2,1,2,3,3,2,1,1


Let's create a copy of the original lagos_df so that we can make the LGA its index. this will make our analysis easier

In [203]:
# First let's make a copy of the lagos_df
copy_lagos_df = lagos_df.copy(deep=True)

# Then let's make LGAs the index
copy_lagos_df.index = copy_lagos_df.LGA

# Let's drop the LGA Column
copy_lagos_df = copy_lagos_df.drop('LGA', axis=1)

# Lets view the effect
copy_lagos_df.head()

,POP,latitude,longitude
LGA,,,
Alimosho,5700714,6.58434,3.257631
Ajeromi-Ifelodun,1746634,6.45512,3.335946
Kosofe,665998,6.58197,3.414836
"Mushin, Lagos",633543,6.53263,3.352022
Oshodi-Isolo,621789,6.54001,3.312415


<h3>Next let's encapsulate all these steps into a single method that we can call</h3>

First let's define a method for plotting each map

In [0]:
def plot_map(lga):
    """ This method takes an LGA name and
    returns a plotted map of it based on lat and lon"""
    lat = copy_lagos_df.loc[lga, 'latitude']
    lon = copy_lagos_df.loc[lga, 'longitude']
    
    # create map of lga using latitude and longitude values
    map_lga = folium.Map(location=[lat, lon], zoom_start=14)

    # add markers to map

    label = "{}".format(lga)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(map_lga)  
    return map_lga

Next let's define a method that returns a filtered out version of up to 200 popular venues within 10KM radius of each LGA

In [0]:
def return_venues(lga):
    
    # let's save the latitude and longitude values
    lat = copy_lagos_df.loc[lga, 'latitude']
    lon = copy_lagos_df.loc[lga, 'longitude']
    
    # First lets save the foursquare credentials
    CLIENT_ID = 'L2XUYCRD44VP2YBDMZ3XY0Z4XKO1JODM5SMG3IIYRZ5330FD' # your Foursquare ID
    CLIENT_SECRET = '5KI00FG4RT2VJQTIDPXFY1N4AOKTIXLIZEAO0AHPP3DPD0U1' # your Foursquare Secret
    VERSION = '20190902'
    
    # Now, let's get the top 200 venues that are in the first entry within a radius of 10 Kilometers.
    radius = 10000 # 10,000 meters is 10KM
    LIMIT=200  # no more than 200 venues
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lon, 
        radius, 
        LIMIT)
    
    # Let's filter out the result
    result = requests.get(url).json()
    
    # From the result, lets get the venues
    venues = result['response']['groups'][0]['items']
    
    # Lets convert the json files to dataframe
    nearby_venues = json_normalize(venues)
    
    # Let's filter the venues and select only columns needed
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance', 'venue.id']
    nearby_venues = nearby_venues.loc[:, filtered_columns]
    
    # Next lets use the clean_category method to clean out category column
    nearby_venues['venue.categories'] = nearby_venues['venue.categories'].apply(clean_category)

    
    return nearby_venues

Let's see a summary of the total number of venues returned for each LGA, as well as the first 5 venues<br>We shall also extract the lgas with less than i5 venues

Always  be aware that the total venues and types returned will be a function of the day and time the code cell below is run.<br> Normally weekends may return fewer official venues than weekdays and sunday mornings in particular return the least amount of available venues.

In [206]:
# first let's create a list of lgas with <= 15 venues to be excluded from our analysis
lga_exclusion_list = []

for i in range(len(copy_lagos_df)):
    x = copy_lagos_df.index[i]
    z = x
    x+='_lga_venues'
    y = return_venues(copy_lagos_df.index[i])

    print('Total number of venues returned for {} is {}'.format(x, len(y)))
    # if total venues returned per lga is less than 15 add to exclusion list
    if len(y) <= 15:
        lga_exclusion_list.append(z)

    print(y.head())
    print()
print('LGAs to be possibly excluded due to low venues count are:- ')
print(lga_exclusion_list)

Total number of venues returned for Alimosho_lga_venues is 39
               venue.name  ...                  venue.id
0           SHAWARMA.COME  ...  4e261fce315161d79d2a8fa5
1             Tantalisers  ...  4ecbf689b6346e2d3a4dead5
2        Wings Restaurant  ...  4f75a8e7e4b06c81e0e3ba05
3      Ipaja market ipaja  ...  51176f6ce4b05931e449abbc
4  De Grange suites & bar  ...  4c93872fa25fef3b4c231223

[5 rows x 6 columns]

Total number of venues returned for Ajeromi-Ifelodun_lga_venues is 59
                        venue.name  ...                  venue.id
0  Mega Chicken Chinese Restaurant  ...  54ce24ae498e6f49a707f987
1                   Rockview Hotel  ...  4e1cbebe7d8b8fe8c30af7b0
2                Film House Cinema  ...  50ccbc9ce4b0d07024dd14f4
3                Pool Golden Tulip  ...  5139ec67e4b06ad169780e29
4            tank and tummy festac  ...  4d4ea5e9ae4e60fce52c02a2

[5 rows x 6 columns]

Total number of venues returned for Kosofe_lga_venues is 79
                       v

Now let's exclude the LGAs with less than 15 venues from our dataframe and future analysis

In [207]:
len(copy_lagos_df)

20

In [0]:
# if we have atleast one lga with less than 15 venues, drop it.
if len(lga_exclusion_list) >= 1:
    copy_lagos_df.drop(index=lga_exclusion_list, inplace=True)

Let's confirm the length

In [209]:
len(copy_lagos_df)

15

Ok, so we now have only lgas left with atleast 15 venues 

Let's see the number of venues returned for the second LGA and the first 5 venues

In [210]:
# first let's confirm the name
copy_lagos_df.index[1]

'Ajeromi-Ifelodun'

In [211]:
Ajeromi_lga_venues = return_venues(copy_lagos_df.index[1])

print('Total number of venues returned is {}'.format(len(Ajeromi_lga_venues)))

Ajeromi_lga_venues.head()

Total number of venues returned is 59


,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.distance,venue.id
0,Mega Chicken Chinese Restaurant,Chinese Restaurant,6.471061,3.312721,3122,54ce24ae498e6f49a707f987
1,Rockview Hotel,Hotel,6.448105,3.360831,2861,4e1cbebe7d8b8fe8c30af7b0
2,Film House Cinema,Multiplex,6.490242,3.357371,4571,50ccbc9ce4b0d07024dd14f4
3,Pool Golden Tulip,Pool,6.465255,3.302554,3861,5139ec67e4b06ad169780e29
4,tank and tummy festac,Fast Food Restaurant,6.477172,3.332893,2477,4d4ea5e9ae4e60fce52c02a2


Let's see the different types of venues present

In [212]:
Ajeromi_cat_count = Ajeromi_lga_venues.groupby('venue.categories').size().to_frame()

Ajeromi_cat_count.columns = ['frequency']

# Let's see the categories of venues present in the alimosho LGA
Ajeromi_cat_count.T

venue.categories,African Restaurant,Art Gallery,Asian Restaurant,Bagel Shop,Bar,Beach,Breakfast Spot,Chinese Restaurant,Cocktail Bar,Convention Center,Cupcake Shop,Department Store,Diner,Fast Food Restaurant,Flea Market,Grocery Store,Gym / Fitness Center,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Lounge,Market,Mediterranean Restaurant,Modern European Restaurant,Movie Theater,Multiplex,Music Venue,Nightclub,Park,Photography Studio,Pizza Place,Pool,Pub,Restaurant,Shopping Mall,Soccer Field,Stables,Supermarket
frequency,5,2,2,1,1,1,1,2,1,1,1,1,1,2,1,1,1,5,1,1,2,3,1,1,1,2,1,1,2,2,1,2,1,1,1,2,1,1,1


Let's plot the map of 'Ajeromi-LgA'

In [213]:
plot_map(copy_lagos_df.index[1])

Next let's see the number of venues returned for the third LGA and the first 5 venues

In [214]:
# First let's confirm the name
copy_lagos_df.index[2]

'Kosofe'

In [215]:
Kosofe_lga_venues = return_venues(copy_lagos_df.index[2])

print('Total number of venues returned is {}'.format(len(Kosofe_lga_venues)))

Kosofe_lga_venues.head()

Total number of venues returned is 79


,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.distance,venue.id
0,De Prince Supermarket,Department Store,6.554255,3.388933,4210,4fce4c1ae4b0d4988249ad4a
1,Mende Suya Spot,BBQ Joint,6.568890,3.371953,4960,4c5af9572091a593d5b160d0
2,Maryland Mall,Shopping Mall,6.567211,3.367269,5511,57767482498e1b23fc39b866
3,Zen Garden Chinese Resturant,Chinese Restaurant,6.585504,3.357131,6393,4cbb0b289552b60c6994dd8b
4,The Londoners Bar & Restaurant,Bar,6.573301,3.357092,6458,4da4afbcb1c9370496db15a2


Let's see the different types of venues present

In [216]:
Kosofe_cat_count = Kosofe_lga_venues.groupby('venue.categories').size().to_frame()

Kosofe_cat_count.columns = ['frequency']

# Let's see the categories of venues present in the alimosho LGA
Kosofe_cat_count.T

venue.categories,African Restaurant,Airport,Airport Food Court,Airport Terminal,American Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Beer Garden,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cupcake Shop,Department Store,Electronics Store,Fast Food Restaurant,Flea Market,Heliport,Hotel,IT Services,Ice Cream Shop,Lounge,Luggage Store,Men's Store,Movie Theater,Multiplex,Nightclub,Performing Arts Venue,Pizza Place,Pool,Restaurant,Shopping Mall,Soup Place,Spa,Steakhouse
frequency,4,1,1,1,1,3,1,2,3,1,1,1,3,1,1,4,1,3,1,6,1,1,3,1,3,2,1,1,1,1,5,1,4,1,3,6,1,1,2


Now let's plot the map of Kosofe LGA

In [217]:
plot_map(copy_lagos_df.index[2])

We can repeat the above steps for all the remaining LGAs, but remember our task is to find the ideal LGA location for a Tech start-up.<br>So we need to be smart about it.<br> I believe we need to get the venues returned by each LGA and a total list of venue categories.<br>Then we compare each LGA venues to the categories of venues we care about.

So first, let's define a method that converts the venue categories in each LGA to small case and adds it to a set. So that we can get a set of all available venue categories in our dataframe. 

The essence of converting each category name to all small case is just incase one category exists in upper and lower case some where. So that it won't be added as 2 categories

In [0]:
def store_venue_categories(df):
    venue_set = set()
    for lga in df.index:
        df_lga = return_venues(lga)
        # Converting each category name to lower case using a list comprehension
        df_lga['venue.categories'] = [x.lower() for x in list(df_lga['venue.categories'])]
        # Add each converted name to a set to avoid any possible duplicates
        for category in df_lga['venue.categories']:
            venue_set.add(category)
    # Finally return a list of all venue categories in our DataFrame.
    return list(venue_set)

In [0]:
all_venue_categories = store_venue_categories(copy_lagos_df)

Let's see all the categories of venues listed in our data set

In [220]:
print('Total venue categories is: {} categories'.format(len(all_venue_categories)))

all_venue_categories

Total venue categories is: 101 categories


['bbq joint',
 'airport food court',
 'performing arts venue',
 'theater',
 'supermarket',
 'grocery store',
 'movie theater',
 'coffee shop',
 'breakfast spot',
 'karaoke bar',
 'art gallery',
 'restaurant',
 'surf spot',
 'spa',
 'neighborhood',
 'african restaurant',
 'diner',
 'pub',
 'bakery',
 'electronics store',
 'airport service',
 'seafood restaurant',
 'sushi restaurant',
 'shopping mall',
 'theme restaurant',
 'gym / fitness center',
 'café',
 'mexican restaurant',
 'comfort food restaurant',
 'campground',
 'dessert shop',
 "men's store",
 'boutique',
 'bistro',
 'nightclub',
 'auto garage',
 'burger joint',
 'pool',
 'multiplex',
 'market',
 'golf course',
 'plaza',
 'sports bar',
 'wine shop',
 'photography studio',
 'playground',
 'gas station',
 'athletics & sports',
 'bar',
 'chinese restaurant',
 'soup place',
 'fast food restaurant',
 'mobile phone shop',
 'luggage store',
 'airport',
 'snack place',
 'cupcake shop',
 'food',
 'bagel shop',
 'italian restaurant',
 '

Next, we need to arrange all categories in order of importance to our analysis. This means categories that are more important be moved up the list, while others are down.<br>
First let's group similar venue types together such as coffee shop and cupcake shop

In [0]:
continental_restaurants = ['italian restaurant', 'modern european restaurant','african restaurant','brazilian restaurant','theme restaurant', 
                           'mexican restaurant',  'seafood restaurant','american restaurant','mediterranean restaurant','fast food restaurant','japanese restaurant',
                           'chinese restaurant','comfort food restaurant','asian restaurant','indian restaurant', 'sushi restaurant','restaurant']

eateries = ['bbq joint', 'fried chicken joint', 'food','burger joint' ,'pizza place','steakhouse','food court','soup place','airport food court','food & drink shop',
            'breakfast spot','diner']

coffee_shops = ['bagel shop','dessert shop', 'café','cupcake shop','coffee shop']

gas_stations = ['gas station']

ice_cream_n_confectionery = ['snack place','bakery','ice cream shop']

hotels_resorts_spas = ['resort','hotel','spa']

bars_n_lounges = ['lounge','pub','bar','sports bar','beer garden', 'karaoke bar','cocktail bar','airport lounge']

airport = ['airport terminal','airport service']

bus_stations = ['bus station']

heliport = ['bistro','heliport']

it_services_n_hubs = ['it services']

shopping_malls_n_stores = ['department store', 'plaza','grocery store','convenience store','supermarket','furniture / home store','shopping mall','wine shop',
                           "men's store"]

gym_sports_facilities_games = ['gym / fitness center','playground','soccer field','arcade','golf course','pool hall','surf spot','pool','baseball stadium',
                               'athletics & sports']

markets = ['farmers market','market','flea market']

fashion_n_clothing = ['boutique','clothing store']

arts_studios_galleries = ['comedy club','art gallery','theater','performing arts venue']

convention_centers = ['multiplex','convention center']

halls_events_venues = ['music venue','campground']

night_clubs = ['nightclub']

beach = ['beach']

parks = ['park']

cinemas = ['movie theater']

auto_services = ['auto garage']

electronics_shop = ['mobile phone shop','electronics store']

residences = ['neighborhood']

other_services = ['stables','photography studio','pier','luggage store','bike rental / bike share']

Let's now create a super list that stores each category list above in a list

In [0]:
super_list  = [continental_restaurants, eateries, coffee_shops, gas_stations, ice_cream_n_confectionery, hotels_resorts_spas, bars_n_lounges,
               airport, bus_stations, heliport, it_services_n_hubs, shopping_malls_n_stores, gym_sports_facilities_games, markets, fashion_n_clothing,
               arts_studios_galleries, convention_centers, halls_events_venues, night_clubs, beach, parks, cinemas, auto_services, electronics_shop,
               residences, other_services]

Let's confirm we have the same number of unique venues as we got using the returned_venues method above

In [223]:
count = 0
for i in super_list:
    count += len(i)
print(count)

100


We had 101 venue categories earlier. So one venue is missing, let's trace it and add it up to the right category, else it will distort our output

In [0]:
if count < len(all_venue_categories):
    test_set = set()

    for i in super_list:
        for j in i:
            test_set.add(j)

    len(test_set)

Now that we have a test_set containing all venue categories in super_list,<br> Let's set it against the all_venue_categories list to identify the missing venue category 

In [0]:
missing_category = set(all_venue_categories).difference(test_set)

In [226]:
# Let's see the missing category

print(missing_category)

{'airport'}


So let's add the airport category to the appropriate list. which should be airport list

In [0]:
airport.append('airport')

In [228]:
print(airport)

['airport terminal', 'airport service', 'airport']


Let's once again confirm that we now have equal number of venues

In [229]:
count = 0
for i in super_list:
    count += len(i)
print(count)

101


Ok, Let's move on to the next step which is creating a DataFrame with categories as index and LGAs as columns.<br>Then each LGA would have a score based on the number of venues in each category.<br>
After that we would sort the data frame by categories we care most about.

In [0]:
# Let's make the categories the index of the new DataFrame

data_index = ['continental_restaurants', 'eateries', 'coffee_shops', 'gas_stations', 'ice_cream_n_confectionery', 'hotels_resorts_spas', 'bars_n_lounges',
               'airport', 'bus_stations', 'heliport', 'it_services_n_hubs', 'shopping_malls_n_stores', 'gym_sports_facilities_games', 'markets', 'fashion_n_clothing',
               'arts_studios_galleries', 'convention_centers', 'halls_events_venues', 'night_clubs', 'beach', 'parks', 'cinemas', 'auto_services', 'electronics_shop',
               'residences', 'other_services']

In [0]:
# Let's create a list of LGAs to be the columns of our new DataFrame

data_columns = list(copy_lagos_df.index)

In [232]:
# Let's create a new DataFrame
summary_df = pd.DataFrame(index= data_index, columns= data_columns)

# Next let's replace any possoble NaN values with 0
summary_df.fillna(0, inplace=True)

# Let's see the first 5 rows
summary_df.head()

,Alimosho,Ajeromi-Ifelodun,Kosofe,"Mushin, Lagos",Oshodi-Isolo,Surulere,Agege,Ifako-Ijaiye,Somolu,Amuwo-Odofin,Lagos Mainland,Ikeja,Eti-Osa,Apapa,Lagos Island
continental_restaurants,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
eateries,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
coffee_shops,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
gas_stations,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ice_cream_n_confectionery,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Now let's create a method that adds the number of venues in each category per LGA to the summary_df DataFrame above

In [0]:
def update_lga_category_values(df_1, df_2):
    """df_1 refers to the copy_lagos_df DataFrame and
    df_2 refers to the summary_df DataFrame,
    This method returns an updated df_2, showing the total number
    of each type of venues that exist in each LGA"""

    # let's make a refence to the super_list object
    new_list = super_list.copy()
    # Let's iterate through the copy_lagos_df DataFrame
    for i in df_1.index:
        # Lets use the return_venues method defined earlier to get the venues for each LGA
        df_lga_venues = return_venues(i)
        # convert the venue.categories elements to all lower case for each LGA using list comp.
        df_lga_venues['venue.categories'] = [x.lower() for x in list(df_lga_venues['venue.categories'])]
        # Let's iterate through each venue category in the given LGA
        for x in df_lga_venues['venue.categories']:
            # Let's check if the venue is in the first element of the new_list, 
            # which is a copy of the Super_list we created earlier
            for y in range(len(new_list)):
                # If it exists add one to the category under which it exists
                if x in new_list[y]:
                    df_2[i][y] += 1
                # if it doesn't exist continue seeking the right category
                else:
                    continue
    # Finally add a Total row that shows the total venues returned per category per LGA
    df_2.loc["Total_Venues"] = df_2.sum()
    return df_2

In [0]:
summary_df = update_lga_category_values(copy_lagos_df, summary_df)

In [235]:
summary_df

,Alimosho,Ajeromi-Ifelodun,Kosofe,"Mushin, Lagos",Oshodi-Isolo,Surulere,Agege,Ifako-Ijaiye,Somolu,Amuwo-Odofin,Lagos Mainland,Ikeja,Eti-Osa,Apapa,Lagos Island
continental_restaurants,10,17,17,21,23,18,20,16,3,4,23,21,9,30,7
eateries,7,4,11,15,16,7,16,11,3,2,7,16,5,7,5
coffee_shops,0,2,4,5,4,2,2,2,0,0,5,2,0,5,0
gas_stations,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
ice_cream_n_confectionery,0,1,5,6,6,1,4,0,1,1,4,4,3,3,3
hotels_resorts_spas,2,5,4,5,7,10,5,3,1,4,9,5,6,11,9
bars_n_lounges,4,6,6,9,10,12,9,4,1,0,15,10,9,13,11
airport,2,0,2,0,0,0,3,0,0,0,0,3,0,0,0
bus_stations,0,0,0,0,0,0,1,2,3,1,0,0,0,0,2
heliport,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0


Let's make an effort to select our top 20 categories from the summary_df above and save that to a list

In [0]:
top_criteria = ['hotels_resorts_spas','airport','it_services_n_hubs','gym_sports_facilities_games','shopping_malls_n_stores','coffee_shops','markets',
                'bars_n_lounges','arts_studios_galleries','halls_events_venues','continental_restaurants','eateries','gas_stations','cinemas','residences',
                'auto_services','convention_centers','parks','electronics_shop','night_clubs']

In [0]:
top_criteria_df = summary_df.loc[top_criteria,]

top_criteria_df.loc['Total Venues'] = top_criteria_df.sum()

In [238]:
top_criteria_df

,Alimosho,Ajeromi-Ifelodun,Kosofe,"Mushin, Lagos",Oshodi-Isolo,Surulere,Agege,Ifako-Ijaiye,Somolu,Amuwo-Odofin,Lagos Mainland,Ikeja,Eti-Osa,Apapa,Lagos Island
hotels_resorts_spas,2,5,4,5,7,10,5,3,1,4,9,5,6,11,9
airport,2,0,2,0,0,0,3,0,0,0,0,3,0,0,0
it_services_n_hubs,0,0,1,1,1,1,0,0,0,0,1,1,0,0,0
gym_sports_facilities_games,1,3,1,5,4,8,3,2,0,1,8,3,4,8,5
shopping_malls_n_stores,5,5,14,18,14,12,15,7,4,1,11,13,8,6,8
coffee_shops,0,2,4,5,4,2,2,2,0,0,5,2,0,5,0
markets,2,2,1,1,2,1,2,2,1,1,1,2,1,1,2
bars_n_lounges,4,6,6,9,10,12,9,4,1,0,15,10,9,13,11
arts_studios_galleries,0,2,1,1,1,2,1,4,0,0,3,1,1,4,1
halls_events_venues,1,1,0,0,0,1,1,2,0,0,1,1,0,1,0


let's check for the LGA that has the fewest number of 0's. This means the LGA that has the most wide spread categories of venues.<br>
Because ideally, we want a location that lacks the least, on the categories of venues we deem important.

In [239]:
# First let's save the columns to a variable
column_list = list(top_criteria_df.columns)

# Let's iterate through each LGA in the top_criteria_df and count the number of missing categories
# Then append each LGA and its missing categories score to the first dictionary

lga_dict = {}  # first dict
min_dict = {}  # second dict

for i in column_list:
    zero_count = 0
    for j in top_criteria_df[i]:
        if j == 0:
            zero_count  += 1
            lga_dict[i] = zero_count
print(lga_dict)

# Let's iterate through the first dictionary and append the keys and values 
# corresponding to the LGAs with the least missing categories to the second dictionary.

for i, j in lga_dict.items():
    if j == min(lga_dict.values()):
        min_dict[i] = j

# show the LGAs with the corresponding lowest missing categories 
print(min_dict)

{'Alimosho': 9, 'Ajeromi-Ifelodun': 6, 'Kosofe': 5, 'Mushin, Lagos': 6, 'Oshodi-Isolo': 5, 'Surulere': 5, 'Agege': 5, 'Ifako-Ijaiye': 7, 'Somolu': 14, 'Amuwo-Odofin': 12, 'Lagos Mainland': 4, 'Ikeja': 4, 'Eti-Osa': 8, 'Apapa': 5, 'Lagos Island': 6}
{'Lagos Mainland': 4, 'Ikeja': 4}


With both Lagos Mainland LGA and Ikeja LGA having the least absent category of 4 venues each.<br>
Plus Lagos Mainland having 95 active venues in 16 top categories and,<br>
Ikeja having 83 active venues in 16 top categories.<br>
It is safe to say Lagos Mainland LGA is the most ideal location in Lagos state to cite an office.<br>
Although a closer look indicates that Ikeja LGA has three airports and that should count for a lot of consideration.<br>
By and Large, Lagos Mainland LGA and Ikeja LGA are the two most ideal locations to cite an office in Lagos State.

The next set of good locations with only 5 missing categories are:<br>
Apapa:- With 96 available venues<br>
Oshodi-Isolo:- with 92 available venues<br>
Surulere:- With 84 available venues<br>

<h3>Now let's analyse the Lagos Mainland LGA</h3>

In [240]:
# Let's see a map of Lagos Mainland LGA

plot_map('Lagos Mainland')

# feel free to zoom in the map

Background<br>
Lagos Mainland was founded by Chief Olofin and is peopled by the same group as found in Lagos especially the Egbas and Aworis. <br>
Lagos Mainland developed from settlements such as Ebute-Metta, Ido-Otto and Ijora.
<br>
<br>
Communities<br>
The communities of Lagos Mainland incude >>
<br>Yaba, <br>Ebute-Metta, <br>Iddo-Otto, <br>Iwaya, <br>Akoka, <br>Makoko, <br>Abule-nla.<br>
See [link](https://web.archive.org/web/20070929061412/http://www.mainlandlg.com/)

In [241]:
# Let's inspect the Lagos Mainland Data frame

lagos_mainland_df = return_venues('Lagos Mainland')

lagos_mainland_df.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.distance,venue.id
0,White House,African Restaurant,6.506143,3.375522,2025,4d344e1098336dcb474236f0
1,Film House Cinema,Multiplex,6.490242,3.357371,3429,50ccbc9ce4b0d07024dd14f4
2,The Banana Island Patisserie,Bagel Shop,6.509970,3.391848,1923,51cabaad498eb06f0fdb0340
3,Ozone Cinemas,Movie Theater,6.506480,3.374521,2130,4de68e0418389f05587cbc03
4,Banilux,Nightclub,6.505259,3.375517,1956,4de00f9a3151d414eb1dcfae


In [242]:
lagos_mainland_df.shape

(100, 6)

#### Create a map of Lagos Mainland with Venues superimposed on top.

In [243]:
address = 'Lagos Mainland'

geolocator = Nominatim(user_agent="NG_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lagos Mainland are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Lagos Mainland are 6.49306105, 3.38824963720956.


In [244]:
map_lagos_mainland_venues = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, venue_category, venue_name in zip(lagos_mainland_df['venue.location.lat'], lagos_mainland_df['venue.location.lng'], lagos_mainland_df['venue.categories'], lagos_mainland_df['venue.name']):
    label = '{}, {}'.format(venue_name, venue_category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lagos_mainland_venues)  
    
map_lagos_mainland_venues

Let's write code to show the top 10 most frequent venue categories in Lagos Mainland

In [245]:
lagos_mainland_df['venue.categories'].value_counts().head(10)

Hotel                   9
Lounge                  7
Shopping Mall           6
African Restaurant      5
Nightclub               4
Bar                     4
Restaurant              3
Art Gallery             3
Department Store        3
Fast Food Restaurant    3
Name: venue.categories, dtype: int64

### Exploring Lagos State LGAs and Venues

In [246]:
lagos_df.head()

,LGA,POP,latitude,longitude
0,Alimosho,5700714,6.58434,3.257631
1,Ajeromi-Ifelodun,1746634,6.45512,3.335946
2,Kosofe,665998,6.58197,3.414836
3,"Mushin, Lagos",633543,6.53263,3.352022
4,Oshodi-Isolo,621789,6.54001,3.312415


##### Let's Create a map of Lagos State with LGAs superimposed on top.

In [247]:
address = 'Lagos'

geolocator = Nominatim(user_agent="NG_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lagos State are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Lagos State are 6.4550575, 3.3941795.


In [248]:
map_lagos_state = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, LGA  in zip(lagos_df['latitude'], lagos_df['longitude'], lagos_df['LGA']):
    label = '{}'.format(LGA)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lagos_state)  
    
map_lagos_state

#### Let's create a method to return a DataFrame of the venues in lagos state in each LGA 

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['LGA', 
                  'LGA_Latitude', 
                  'LGA_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)

In [250]:
lagos_state_venues = getNearbyVenues(names= lagos_df['LGA'],
                                     latitudes= lagos_df['latitude'],
                                     longitudes= lagos_df['longitude'])

Alimosho
Ajeromi-Ifelodun
Kosofe
Mushin, Lagos
Oshodi-Isolo
Ojo
Ikorodu
Surulere
Agege
Ifako-Ijaiye
Somolu
Amuwo-Odofin
Lagos Mainland
Ikeja
Eti-Osa
Badagry
Apapa
Lagos Island
Epe
Ibeju-Lekki


#### Let's check the size of the resulting dataframe and the first 5 rows.

In [251]:
print(lagos_state_venues.shape)

lagos_state_venues

(1089, 7)


,LGA,LGA_Latitude,LGA_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Alimosho,6.584343,3.257631,SHAWARMA.COME,6.604609,3.277413,Pizza Place
1,Alimosho,6.584343,3.257631,Tantalisers,6.649299,3.265609,Burger Joint
2,Alimosho,6.584343,3.257631,Wings Restaurant,6.576023,3.325612,Restaurant
3,Alimosho,6.584343,3.257631,Ipaja market ipaja,6.602248,3.255585,Market
4,Alimosho,6.584343,3.257631,De Grange suites & bar,6.602309,3.267038,Bar
...,...,...,...,...,...,...,...
1084,Epe,6.524750,4.057246,Jubilee Chalets,6.582906,3.989036,Hotel
1085,Ibeju-Lekki,6.471125,3.814750,Eleko Beach,6.441165,3.854528,Beach
1086,Ibeju-Lekki,6.471125,3.814750,lakowe lakes golfcourt,6.459769,3.743862,Golf Course
1087,Ibeju-Lekki,6.471125,3.814750,Omu Resort,6.489678,3.753977,Resort


#### Let's check how many venues were returned for each neighborhood

In [252]:
lagos_state_venues.groupby('LGA').count()

,LGA_Latitude,LGA_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
LGA,,,,,,
Agege,91,91,91,91,91,91
Ajeromi-Ifelodun,59,59,59,59,59,59
Alimosho,39,39,39,39,39,39
Amuwo-Odofin,20,20,20,20,20,20
Apapa,100,100,100,100,100,100
Badagry,5,5,5,5,5,5
Epe,2,2,2,2,2,2
Eti-Osa,54,54,54,54,54,54
Ibeju-Lekki,4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [253]:
print('There are {} unique categories of venues returned for Lagos State.'.format(lagos_state_venues['Venue_Category'].nunique()))

There are 105 unique categories of venues returned for Lagos State.


#### Let's return the top 10 most frequent venues for each LGA

First let's onehot encode the DataFrame

In [254]:
# one hot encoding
lagos_onehot = pd.get_dummies(lagos_state_venues[['Venue_Category']], prefix="", prefix_sep="")

# add LGA column back to dataframe
lagos_onehot['LGA'] = lagos_state_venues['LGA'] 

# move LGA column to the first column
fixed_columns = [lagos_onehot.columns[-1]] + list(lagos_onehot.columns[:-1])
lagos_onehot = lagos_onehot[fixed_columns]

lagos_onehot.head()

,LGA,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Beach,Beer Garden,Bike Rental / Bike Share,Bistro,Boutique,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Station,Café,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Convention Center,Cupcake Shop,Department Store,Dessert Shop,...,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Other Great Outdoors,Park,Performing Arts Venue,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Resort,Restaurant,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soup Place,Spa,Sports Bar,Stables,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Theater,Theme Restaurant,Wine Shop
0,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [255]:
lagos_onehot.shape

(1089, 106)

Next, let's group rows by LGA and by taking the mean of the frequency of occurrence of each category

In [256]:
lagos_grouped = lagos_onehot.groupby('LGA').mean().reset_index()

lagos_grouped

,LGA,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Beach,Beer Garden,Bike Rental / Bike Share,Bistro,Boutique,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Station,Café,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Convention Center,Cupcake Shop,Department Store,Dessert Shop,...,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Other Great Outdoors,Park,Performing Arts Venue,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Resort,Restaurant,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soup Place,Spa,Sports Bar,Stables,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Theater,Theme Restaurant,Wine Shop
0,Agege,0.032967,0.021978,0.010989,0.010989,0.010989,0.000000,0.010989,0.000000,0.000000,0.000000,0.010989,0.00,0.032967,0.000000,0.010989,0.032967,0.00,0.000000,0.010989,0.00,0.000000,0.010989,0.00,0.000000,0.032967,0.010989,0.010989,0.010989,0.032967,0.010989,0.000000,0.010989,0.000000,0.000000,0.043956,0.000000,0.000000,0.010989,0.00,...,0.010989,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.010989,0.0,0.000000,0.000000,0.021978,0.0,0.021978,0.010989,0.000000,0.000000,0.054945,0.000000,0.00,0.021978,0.000000,0.000000,0.00,0.043956,0.000000,0.109890,0.000000,0.000000,0.010989,0.010989,0.000000,0.000000,0.021978,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,Ajeromi-Ifelodun,0.084746,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033898,0.033898,0.000000,0.00,0.000000,0.016949,0.000000,0.016949,0.00,0.016949,0.000000,0.00,0.000000,0.000000,0.00,0.016949,0.000000,0.000000,0.000000,0.000000,0.033898,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.016949,0.016949,0.016949,0.00,...,0.016949,0.016949,0.000000,0.00,0.00,0.016949,0.033898,0.016949,0.0,0.016949,0.000000,0.033898,0.0,0.033898,0.000000,0.016949,0.000000,0.033898,0.000000,0.00,0.016949,0.000000,0.016949,0.00,0.016949,0.000000,0.033898,0.000000,0.016949,0.000000,0.000000,0.000000,0.016949,0.000000,0.016949,0.000000,0.000000,0.000000,0.00,0.000000
2,Alimosho,0.000000,0.025641,0.025641,0.025641,0.025641,0.000000,0.025641,0.000000,0.000000,0.000000,0.025641,0.00,0.000000,0.000000,0.000000,0.025641,0.00,0.000000,0.000000,0.00,0.000000,0.025641,0.00,0.000000,0.051282,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.025641,0.00,...,0.025641,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.051282,0.000000,0.000000,0.000000,0.076923,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.076923,0.000000,0.051282,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.025641
3,Amuwo-Odofin,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.00,0.050000,0.000000,0.05,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.05,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.050000,0.000000,0.000000,0.05,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Apapa,0.090000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.030000,0.010000,0.010000,0.00,0.000000,0.010000,0.000000,0.010000,0.01,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.010000,0.010000,0.000000,0.020000,0.000000,0.020

Let's confirm the new shape

In [257]:
lagos_grouped.shape

(20, 106)

Let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each LGA in Lagos

In [259]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['LGA']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
lga_venues_sorted = pd.DataFrame(columns=columns)
lga_venues_sorted['LGA'] = lagos_grouped['LGA']

for ind in np.arange(lagos_grouped.shape[0]):
    lga_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lagos_grouped.iloc[ind, :], num_top_venues)

lga_venues_sorted

,LGA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agege,Shopping Mall,Fast Food Restaurant,Pizza Place,Convenience Store,Hotel,Lounge,Restaurant,Ice Cream Shop,Chinese Restaurant,Burger Joint
1,Ajeromi-Ifelodun,African Restaurant,Hotel,Lounge,Shopping Mall,Nightclub,Park,Pizza Place,Japanese Restaurant,Fast Food Restaurant,Movie Theater
2,Alimosho,Fast Food Restaurant,Pizza Place,Restaurant,Burger Joint,Lounge,Park,Shopping Mall,Hotel,Wine Shop,Farmers Market
3,Amuwo-Odofin,Hotel,Park,Fast Food Restaurant,African Restaurant,Shopping Mall,Chinese Restaurant,Mobile Phone Shop,Bike Rental / Bike Share,Beach,Pool
4,Apapa,Hotel,African Restaurant,Lounge,Restaurant,Nightclub,Japanese Restaurant,Ice Cream Shop,Art Gallery,Pizza Place,Diner
5,Badagry,History Museum,Museum,Fast Food Restaurant,Beer Garden,Wine Shop,Cupcake Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop
6,Epe,Hotel,Other Great Outdoors,Wine Shop,Department Store,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club
7,Eti-Osa,Lounge,Hotel,Bar,Beach,Ice Cream Shop,Fast Food Restaurant,Gym / Fitness Center,Pizza Place,Convenience Store,Department Store
8,Ibeju-Lekki,Golf Course,Arcade,Beach,Resort,Department Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club
9,Ifako-Ijaiye,Fast Food Restaurant,Shopping Mall,Pizza Place,Hotel,Restaurant,Burger Joint,BBQ Joint,Chinese Restaurant,Bus Station,Bar


We can clearly see that hotels are the most common venues in majority of LGA areas in Lagos State.<br>Other common venues are fast food restaurants and shopping malls

#### Let's Cluster Neighborhoods in Lagos State by LGAs

Run k-means to cluster the State into 5 clusters.

In [279]:
# set number of clusters
kclusters = 5

lagos_grouped_clustering = lagos_grouped.drop('LGA', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lagos_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 3, 2, 0, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
      dtype=int32)

Let's update the lga_venues_sorted DataFrame to include the cluster labels, as well as lat and long data per LGA

In [261]:
lagos_df.tail()

,LGA,POP,latitude,longitude
15,Badagry,241437,6.43873,2.907486
16,Apapa,217661,6.44519,3.368373
17,Lagos Island,209665,6.51486,3.490866
18,Epe,181715,6.52475,4.057246
19,Ibeju-Lekki,117542,6.47113,3.814750


In [0]:
# insert clustering labels
lga_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# Let's select LGA, latitude and longitude data columns from lagos_df
lagos_merged = lagos_df[['LGA', 'latitude', 'longitude']]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
lagos_merged = lagos_merged.join(lga_venues_sorted.set_index('LGA'), on='LGA')

In [263]:
lagos_merged

,LGA,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alimosho,6.58434,3.257631,0,Fast Food Restaurant,Pizza Place,Restaurant,Burger Joint,Lounge,Park,Shopping Mall,Hotel,Wine Shop,Farmers Market
1,Ajeromi-Ifelodun,6.45512,3.335946,0,African Restaurant,Hotel,Lounge,Shopping Mall,Nightclub,Park,Pizza Place,Japanese Restaurant,Fast Food Restaurant,Movie Theater
2,Kosofe,6.58197,3.414836,0,Shopping Mall,Fast Food Restaurant,Nightclub,African Restaurant,Convenience Store,Pizza Place,Ice Cream Shop,Bar,BBQ Joint,Restaurant
3,"Mushin, Lagos",6.53263,3.352022,0,Shopping Mall,Fast Food Restaurant,Pizza Place,African Restaurant,Nightclub,Department Store,Lounge,Hotel,Convenience Store,Chinese Restaurant
4,Oshodi-Isolo,6.54001,3.312415,0,Fast Food Restaurant,Shopping Mall,African Restaurant,Hotel,Pizza Place,Bar,Nightclub,Department Store,Lounge,Chinese Restaurant
5,Ojo,1.09584,-76.611069,1,Hotel,Restaurant,Park,Wine Shop,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club
6,Ikorodu,6.61912,3.504127,0,Hotel Bar,African Restaurant,Bar,Mobile Phone Shop,Comedy Club,Café,Pizza Place,Beach,Bus Station,Art Gallery
7,Surulere,6.4872,3.353259,0,Hotel,African Restaurant,Lounge,Shopping Mall,Bar,Department Store,Nightclub,Pizza Place,Fast Food Restaurant,Art Gallery
8,Agege,6.62526,3.311209,0,Shopping Mall,Fast Food Restaurant,Pizza Place,Convenience Store,Hotel,Lounge,Restaurant,Ice Cream Shop,Chinese Restaurant,Burger Joint
9,Ifako-Ijaiye,6.6636,3.289491,0,Fast Food Restaurant,Shopping Mall,Pizza Place,Hotel,Restaurant,Burger Joint,BBQ Joint,Chinese Restaurant,Bus Station,Bar


####  let's visualize the resulting clusters


In [271]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lagos_merged['latitude'], lagos_merged['longitude'], lagos_merged['LGA'], lagos_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Inspecting The Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

We can inspect Cluster One closely

In [273]:
cluster_one = lagos_merged[lagos_merged['Cluster Labels'] == 0]
cluster_one

,LGA,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alimosho,6.58434,3.257631,0,Fast Food Restaurant,Pizza Place,Restaurant,Burger Joint,Lounge,Park,Shopping Mall,Hotel,Wine Shop,Farmers Market
1,Ajeromi-Ifelodun,6.45512,3.335946,0,African Restaurant,Hotel,Lounge,Shopping Mall,Nightclub,Park,Pizza Place,Japanese Restaurant,Fast Food Restaurant,Movie Theater
2,Kosofe,6.58197,3.414836,0,Shopping Mall,Fast Food Restaurant,Nightclub,African Restaurant,Convenience Store,Pizza Place,Ice Cream Shop,Bar,BBQ Joint,Restaurant
3,"Mushin, Lagos",6.53263,3.352022,0,Shopping Mall,Fast Food Restaurant,Pizza Place,African Restaurant,Nightclub,Department Store,Lounge,Hotel,Convenience Store,Chinese Restaurant
4,Oshodi-Isolo,6.54001,3.312415,0,Fast Food Restaurant,Shopping Mall,African Restaurant,Hotel,Pizza Place,Bar,Nightclub,Department Store,Lounge,Chinese Restaurant
6,Ikorodu,6.61912,3.504127,0,Hotel Bar,African Restaurant,Bar,Mobile Phone Shop,Comedy Club,Café,Pizza Place,Beach,Bus Station,Art Gallery
7,Surulere,6.4872,3.353259,0,Hotel,African Restaurant,Lounge,Shopping Mall,Bar,Department Store,Nightclub,Pizza Place,Fast Food Restaurant,Art Gallery
8,Agege,6.62526,3.311209,0,Shopping Mall,Fast Food Restaurant,Pizza Place,Convenience Store,Hotel,Lounge,Restaurant,Ice Cream Shop,Chinese Restaurant,Burger Joint
9,Ifako-Ijaiye,6.6636,3.289491,0,Fast Food Restaurant,Shopping Mall,Pizza Place,Hotel,Restaurant,Burger Joint,BBQ Joint,Chinese Restaurant,Bus Station,Bar
10,Somolu,6.58333,3.466667,0,Bus Station,Pizza Place,Convenience Store,Department Store,Fast Food Restaurant,Snack Place,Beach,Market,Bar,Restaurant


We can inspect Cluster Two closely

In [274]:
cluster_two = lagos_merged[lagos_merged['Cluster Labels'] == 1]
cluster_two

,LGA,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Ojo,1.09584,-76.611069,1,Hotel,Restaurant,Park,Wine Shop,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club


We can inspect Cluster Three closely

In [275]:
cluster_three = lagos_merged[lagos_merged['Cluster Labels'] == 2]
cluster_three

,LGA,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Epe,6.52475,4.057246,2,Hotel,Other Great Outdoors,Wine Shop,Department Store,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club


We can inspect Cluster Four closely

In [276]:
cluster_four = lagos_merged[lagos_merged['Cluster Labels'] == 3]
cluster_four

,LGA,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Badagry,6.43873,2.907486,3,History Museum,Museum,Fast Food Restaurant,Beer Garden,Wine Shop,Cupcake Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop


We can inspect Cluster Five closely

In [277]:
cluster_five = lagos_merged[lagos_merged['Cluster Labels'] == 4]
cluster_five

,LGA,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Ibeju-Lekki,6.47113,3.81475,4,Golf Course,Arcade,Beach,Resort,Department Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club


### A possible shortfall of this exercise.<br>

You may have noticed that for some reason no educational institution or religious institution was returned by foursquare API. This is a flaw as Lagos has over 50 institutions of learning spread across the state.<br>
Another flaw is on the data for Tech Hubs cited generally around Yaba, which is still under Lagos Mainland LGA.<br>
This adds credence to the fact yet again, that Lagos Mainland LGA is perhaps the most ideal location to cite an office in Lagos State.<br>
Thank you

In [0]:
# uncomment and run code below to kill the notebook and free up space

# import os, signal
# os.kill(os.getpid(),signal.SIGKILL)